In [75]:
import boa
import sys, os
from eth_utils import to_checksum_address
from boa.contracts.abi.abi_contract import ABIContractFactory
from eth_account.account import Account, LocalAccount

parent_dir = os.path.abspath('..')
sys.path.insert(0, parent_dir)

dotenv_path = os.path.expanduser('~/.web3env') # Path to .env file containing rpcs and keys

def load_contract(contract_address, name=None, api_key=os.environ.get("ETHERSCAN_API_KEY")):
    addr = to_checksum_address(contract_address)
    abi = boa.explorer.fetch_abi_from_etherscan(addr, api_key=api_key)
    return ABIContractFactory.from_abi_dict(abi, name=name).at(addr)

rpc_url = os.getenv("ETH_RPC_URL")
assert rpc_url is not None, "Provider url is not set"
# fork the chain
boa.env.fork(url=rpc_url, block_identifier=20420069)
boa.env.enable_fast_mode()
print(f'Forked the chain on block {boa.env.evm.vm.state.block_number}')

addresses_dict = {
    # "pufETH":       '0xd9a442856c234a39a81a089c06451ebaa4306a72', # does not have version() implemented
    "pufETHwstE":   '0xeeda34a377dd0ca676b9511ee1324974fa8d980d',
    "crvUSD":       '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'
}

topholders = {
    "pufETH":       '0xF047ab4c75cebf0eB9ed34Ae2c186f3611aEAfa6',
    "pufETHwstE":   '0xf4FA0C7833e778fB9FB392eC36217e17C9133976',
    "crvUSD":       '0xA920De414eA4Ab66b97dA1bFE9e6EcA7d4219635'
}

spender = '0xce6431d21e3fb1036ce9973a3312368ed96f5ce7' # some unrelated pool (FRAXsDAI)
spender_sc = load_contract(spender, name='FRAXsDAI')

# Convert addresses to checksum format
for key in addresses_dict.keys():
    addresses_dict[key] = to_checksum_address(addresses_dict[key])
# Fetch contract data from Etherscan and create the contracts dictionary

token_contracts = {key: load_contract(address, name=key) for key, address in addresses_dict.items()}

boa.env.set_random_seed(420)
alice = Account.create()
bob = boa.env.generate_address()
charlie = boa.env.generate_address()

# Give Alice eth and tokens
boa.env.set_balance(alice.address, boa.env.get_balance(alice.address) + 1*10**18)
for token in addresses_dict.keys():
    with boa.env.prank(topholders[token]):
        token_contracts[token].transfer(alice.address, 1*10**18)
    print(f'Alice has {token_contracts[token].balanceOf(alice.address)} {token}')


Forked the chain on block 20420069
Alice has 1000000000000000000 pufETHwstE
Alice has 1000000000000000000 crvUSD


In [76]:
token_contracts['crvUSD'].version()

'v1.0.0'

In [77]:
# firsts check that allowance is 0
for token in addresses_dict.keys():
    allowance = token_contracts[token].allowance(alice.address, bob)
    assert allowance == 0
    print(f'Allowance for {bob} is {allowance} for {token}')

Allowance for 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 is 0 for pufETHwstE
Allowance for 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 is 0 for crvUSD


In [83]:
from eip712 import EIP712Message
from eth_account._utils.signing import to_bytes32


def permit_class(contract) -> type[EIP712Message]:
    class Permit(EIP712Message):
        # EIP-712 Domain Fields
        _name_: "string" = contract.name()
        _version_: "string" = contract.version()
        _chainId_: "uint256" = boa.env.evm.chain.chain_id
        _verifyingContract_: "address" = contract.address
        # _salt_: "bytes32" = contract.salt()

        # EIP-2612 Data Fields
        owner: "address"
        spender: "address"
        value: "uint256"
        nonce: "uint256"
        deadline: "uint256"

    return Permit

# now try to issue permit so that Bob can transfer Alice's tokens
for token in addresses_dict.keys():
    print(f'Permitting {bob} to spend {token} ({addresses_dict[token]}) on behalf of {alice.address}')
    # first generate signatiure from alice
    value = 10**19
    nonce = token_contracts[token].nonces(alice.address)
    deadline = 2**256 - 1
    permit = permit_class(token_contracts[token])(owner=alice.address, spender=bob, value=value, nonce=nonce, deadline=deadline)
    sig = alice.sign_message(permit.signable_message)
    print(sig)
    # # now charlie calls permit with acquired signature
    # with boa.env.prank(bob):
    #     tx = token_contracts[token].permit(alice.address, bob, value, deadline, sig.v, to_bytes32(sig.r), to_bytes32(sig.s))
    # assert token_contracts[token].allowance(alice, bob) > 0
    # print(f'Allowance for {bob} is {token_contracts[token].allowance(alice, bob)} for {token}')


Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xee902ce1e83929ee627bbf81d0024f8cfd199d36f032f90877735e7914aaf0a8'), r=34618543039810489027197428548918032459551400793991137651061405453686836940868, s=52072594851280632552069287536463377496359401870852203685177110959482533491059, v=28, signature=HexBytes('0x4c8966a7b605985d85d420b7ee45a439b3b262694f12cabce047e918657dd04473200b530bd69042da185b90440ab243aeb141ec48effdecfb09bd0d7622b1731c'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x0c7d59aa5ec88bce40fba6d278e0de4b8eb3afaa0e2ca12d162f40c2b4a441d8'), r=20657776025325331898167512367821060533991862875254759872840869826151792340353, s=531314255182567166153346495015144

In [103]:
from eip712 import EIP712Message
from eth_account._utils.signing import to_bytes32


def permit_class(contract) -> type[EIP712Message]:
    class Permit(EIP712Message):
        # EIP-712 Domain Fields
        _name_: "string" = contract.name()
        _version_: "string" = contract.version()
        _chainId_: "uint256" = boa.env.evm.chain.chain_id
        _verifyingContract_: "address" = contract.address
        # _salt_: "bytes32" = contract.salt()
        # _salt_: "bytes32" = 0x0000000000000000000000000000000000000000000000000000000000000000

        # EIP-2612 Data Fields
        owner: "address"
        spender: "address"
        value: "uint256"
        nonce: "uint256"
        deadline: "uint256"

    return Permit

# now try to issue permit so that Bob can transfer Alice's tokens
for token in addresses_dict.keys():
    print(f'Permitting {bob} to spend {token} ({addresses_dict[token]}) on behalf of {alice.address}')
    # first generate signatiure from alice
    value = 10**19
    nonce = token_contracts[token].nonces(alice.address)
    deadline = 2**256 - 1
    permit = permit_class(token_contracts[token])(owner=alice.address, spender=bob, value=value, nonce=nonce, deadline=deadline)
    sig = alice.sign_message(permit.signable_message)
    print(sig)
    # # now charlie calls permit with acquired signature
    # with boa.env.prank(bob):
    #     tx = token_contracts[token].permit(alice.address, bob, value, deadline, sig.v, to_bytes32(sig.r), to_bytes32(sig.s))
    # assert token_contracts[token].allowance(alice, bob) > 0
    # print(f'Allowance for {bob} is {token_contracts[token].allowance(alice, bob)} for {token}')


Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xee902ce1e83929ee627bbf81d0024f8cfd199d36f032f90877735e7914aaf0a8'), r=34618543039810489027197428548918032459551400793991137651061405453686836940868, s=52072594851280632552069287536463377496359401870852203685177110959482533491059, v=28, signature=HexBytes('0x4c8966a7b605985d85d420b7ee45a439b3b262694f12cabce047e918657dd04473200b530bd69042da185b90440ab243aeb141ec48effdecfb09bd0d7622b1731c'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x0c7d59aa5ec88bce40fba6d278e0de4b8eb3afaa0e2ca12d162f40c2b4a441d8'), r=20657776025325331898167512367821060533991862875254759872840869826151792340353, s=531314255182567166153346495015144

In [ ]:
# zero salt
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xbd82cf843521e51375965822e267447c5f6df30969af3615bb02bf8c4ba7c776'), r=64688705467306757939225666310195026747702377894275261053515253413651391819876, s=47667337382710111875553003968261248664436068156442405398763344341041314710619, v=27, signature=HexBytes('0x8f048281fe50481894c1ae3310d7f2d4f572cec3db16546f3d540a9842bdb0646962c1c211832f6a4f2a0fee244c10341fafa60b6dbff1200a9e152d91f2c45b1b'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x75f0e40453462b6c4d4d66b6ec10709238c56e1d066428aabad1f109d485785a'), r=15127770303632183756561023058601053577690475949876320223032364837480518131975, s=6878384804131894463586995564078724922005220757433790155649408326746141663936, v=27, signature=HexBytes('0x217203ba906ec4347e2e2a1d5d36aa3ddbc7ce9775e3e72a5278bd93a4e821070f350720390a0599a9d6312350ca73fe0f90e3615f7f23675bde522ec574a2c01b'))
# no salt
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xee902ce1e83929ee627bbf81d0024f8cfd199d36f032f90877735e7914aaf0a8'), r=34618543039810489027197428548918032459551400793991137651061405453686836940868, s=52072594851280632552069287536463377496359401870852203685177110959482533491059, v=28, signature=HexBytes('0x4c8966a7b605985d85d420b7ee45a439b3b262694f12cabce047e918657dd04473200b530bd69042da185b90440ab243aeb141ec48effdecfb09bd0d7622b1731c'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x0c7d59aa5ec88bce40fba6d278e0de4b8eb3afaa0e2ca12d162f40c2b4a441d8'), r=20657776025325331898167512367821060533991862875254759872840869826151792340353, s=53131425518256716615334649501514455942316849076815454398030022837281371595057, v=27, signature=HexBytes('0x2dabe30aed35932e53a878ebe8e4eadc0f2bbd5cf5c9d312f2e1997da20d6d81757752374565c7a34c9af1a5a94c47d3db8df9de78005ad6529310d7c44221311b'))

#real salt
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xb824e987e5c5b5bab350984089f4250df1c535a76f674a5c84d2e158367ba24c'), r=16913679051406756091707278186203229515540451115492973805224540520250531380954, s=16073716070481162119102564797471402863711086700350853286538392988532342833967, v=28, signature=HexBytes('0x2564cd97aeabd2985fd45692c374b9bf93d62e1de59e9af4fed99508e873feda238966a1215ca2ca5e06679aa720d5f1126c3f59b85cd6b63952557b1300c32f1c'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x6278ef46e07f2739009e33a0bd131b972d1e4bfa88916223250c6a0506714975'), r=17684157032342460150062863884227840657362884554483765915895271738470266993885, s=57863609876453660882495215218884385097723183593352757503743980928177804902715, v=28, signature=HexBytes('0x2718e0d057432bc75147f3d4f9fbc1188610564e9221a9a88807bc9e318e7cdd7feda4808c14de345946b0be62146d342d484f8ce55d7e5367858b0e98d7813b1c'))

# b0x0
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend pufETHwstE (0xEEda34A377dD0ca676b9511EE1324974fA8d980D) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0xb66f6b61dd3458c6869f1fe8ef13ac860ac0869cc328eb0401a36f46f0e7267c'), r=96899972275616670358133324374390666697895978227102012319194926178424294422153, s=49167280907141680069076095708398939162725225623262255050368327502663518396840, v=27, signature=HexBytes('0xd63b70d3a95d482d9fa7679b077dd5ace83c72b703ca3508a45069d6a9db36896cb3b1df8890ecf80be48c1461eb54a24af6918b7498a1d7f509ba0a9c0221a81b'))
Permitting 0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945 to spend crvUSD (0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E) on behalf of 0xa89D2d3Bdc5B4339277D80D2e5144d993aB0f2d2
SignedMessage(message_hash=HexBytes('0x0f17ebda60fed6c0b8848bd01a85a511dafc3a913f6edc0bb6bede3eda540688'), r=112235632458876750731527614130276294660737912014602085573573489191605611403454, s=19590206560179300550259597080293657706394020198259854429258757612783986107755, v=27, signature=HexBytes('0xf8231de66d762bd1ee74f12e254dc0f7718b3960d4d1856e471810cce38618be2b4fa9faa99c4f1c43071b53691cec8bfc976432baba0fc3e117086035b1656b1b'))


In [99]:
0x0000000000000000000000000000000000000000000000000000000000000000

0